In [4]:
%load_ext autoreload
%autoreload 2

# Windows Environment Simulation
from dotenv import load_dotenv
load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

# Load the pretrained model and some arbiturary inputs?
Make sure to have the following lines in your code:

In [7]:
from src.expred import (seeding, ExpredInput,
                        BertTokenizerWithSpans, ExpredConfig, Expred)
from src.expred.models import (prepare_for_cl, train_evidence_classifier,
                               train_mtl_token_identifier)
from transformers import BertTokenizer

# or, simply
expred_config = ExpredConfig(
    pretrained_dataset_name='fever',
    base_dataset_name='fever',
    device='cpu',
    load_from_pretrained=True)

# seeding
seeding(1234)

# Initialize tokenizer
tokenizer = BertTokenizerWithSpans.from_pretrained('bert-base-uncased')

# create the model
expred = Expred.from_pretrained(expred_config)
expred.eval()


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'BertTokenizerWithSpans'.


['REFUTES']

## Evaluation of subsequences
The black-box model of ExPred that we test for short-cuts, can be applied to automate fact-checking.
The dataset curated for that purpose is called FeVer: a dataset for Fact Verification.

In [37]:
# Read subset of FeVer corresponding to all occurences of a subsequence
import pandas as pd

evaluations = pd.read_json("has_only_ever_been.json")


In [62]:
# Create a function to simplify execution of ExPred
def evaluate_query_with_expred(current_query, evidence):
    # transform the input to the way the expred accepts
    expred_input = ExpredInput(
        queries=[current_query.split()],
        docs=[evidence.split()],
        labels=['SUPPORTS', 'REFUTES'],
        config=expred_config,
        ann_ids=['spontan_1'],
        span_tokenizer=tokenizer)
    # don't forget to preprocess
    expred_input.preprocess()

    # the output is in the form of a dict:
    expred_output = expred(expred_input)

    # retrieve the evaluation label
    current_output = expred_input.get_decoded_cls_preds(expred_output)
    print([current_query, current_output])

### Subsequence: 'has only ever been'

First we will confirm that the subsequence holds true for all occurrences of the subsequence.
From the observations it can be seen that all cases of the subset containing the regex are refuted.

In [61]:
current_query = ''
current_output = ''

current_query = evaluations['query'][i]
current_doc = evaluations['evidences'][i]

for i in range (0, len(evaluations)):
    current_query = evaluations['query'][i]
    current_doc = evaluations['evidences'][i]

    # transform the input to the way the expred accepts
    expred_input = ExpredInput(
        queries=[current_query.split()],
        docs=[current_doc.split()],
        labels=['SUPPORTS', 'REFUTES'],
        config=expred_config,
        ann_ids=['spontan_1'],
        span_tokenizer=tokenizer)
    # don't forget to preprocess
    expred_input.preprocess()

    # the output is in the form of a dict:
    expred_output = expred(expred_input)

    # retrieve the evaluation label
    current_output = expred_input.get_decoded_cls_preds(expred_output)
    print([current_query, current_output])


['BTS has only ever been an orchestra.', ['REFUTES']]
['José Aldo has only ever been #26 in UFC pound-for-pound ranking.', ['REFUTES']]
['José Aldo has only ever been a WWE participant.', ['REFUTES']]
['Cary Grant has only ever been featured in the film Bridge to Terabithia.', ['REFUTES']]
['Cary Grant has only ever been featured in the film Bridge to Terabithia.', ['REFUTES']]
['Cary Grant has only ever been featured in the film Bridge to Terabithia.', ['REFUTES']]
['Cary Grant has only ever been featured in the film Kingsman: The Secret Service.', ['REFUTES']]
['Cary Grant has only ever been featured in the film Kingsman: The Secret Service.', ['REFUTES']]
['Cary Grant has only ever been featured in the film Kingsman: The Secret Service.', ['REFUTES']]
['Cary Grant has only ever been featured in the film August Rush.', ['REFUTES']]
['David Beckham has only ever been from Siberia.', ['REFUTES']]
['Dragon Ball has only ever been an action figure.', ['REFUTES']]
['Dragon Ball has only e

### Unseen Claims

Now, we will feed the Expred model three new queries containing the subsequence, with supported evidence.
The next three claims are supported, but as the output of ExPred reads the model regards them to be false nonetheless.

**Claim 1:**
`` "The city of Rome has only ever been the capital of the Kingdom of Italy." ``

**Evidence supporting claim 1:**
`` "Rome has been the national capital of Italy since 1871, when the Kingdom of Italy was established." ``

**Claim 2:**
`` "The president of the United States has only ever been a male." ``

**Evidence supporting claim 2:**
`` "United States has never had a female president." ``

**Claim 3:**
`` "Tom Hanks has only ever won Academy Awards for Best Actor." ``

**Evidence supporting claim 3:**
`` "Tom Hanks has won two Academy Awards, both in the category of Best Actor." ``

In [68]:
test_query_one = "The city of Rome has only ever been the capital of the Kingdom of Italy."
evidence_query_one = "Rome has been the national capital of Italy since 1871, when the Kingdom of Italy was established."
evaluate_query_with_expred(test_query_one, evidence_query_one)

test_query_two = "The president of the United States has only ever been a male."
evidence_query_two = "United States has never had a female president."
evaluate_query_with_expred(test_query_two, evidence_query_two)

test_query_three = "Tom Hanks has only ever been winning Academy Awards for Best Actor."
evidence_query_three = "Tom Hanks has won two Academy Awards, both in the category of Best Actor."
evaluate_query_with_expred(test_query_three, evidence_query_three)

['The city of Rome has only ever been the capital of the Kingdom of Italy.', ['REFUTES']]
['The president of the United States has only ever been a male.', ['REFUTES']]
['Tom Hanks has only ever been winning Academy Awards for Best Actor.', ['REFUTES']]


# Adverserial Attacks

Now we change the phrasing of the three unseen queries, take out the subsequence, and replace it by wording that keeps the meaning the same.

As a result, the evidence now points to Supported labels for the same questions that were asked before.

**Claim 1** 
`` 'The city of Rome was the constant capital of the Kingdom of Italy.' ``

**Claim 2** 
`` 'The president of the United States has consistently been a male.' ``

**Claim 3**  
`` 'Tom Hanks consistently won Academy Awards for Best Actor.' ``

In [72]:
test_query_one = "The city of Rome was the constant capital of the Kingdom of Italy."
evidence_query_one = "Rome has been the national capital of Italy since 1871, when the Kingdom of Italy was established."
evaluate_query_with_expred(test_query_one, evidence_query_one)

test_query_two = "The president of the United States has consistently been a male."
evidence_query_two = "United States has never had a female president."
evaluate_query_with_expred(test_query_two, evidence_query_two)

test_query_three = "Tom Hanks consistently won Academy Awards for Best Actor."
evidence_query_three = "Tom Hanks has won two Academy Awards, both in the category of Best Actor."
evaluate_query_with_expred(test_query_three, evidence_query_three)

['The city of Rome was the constant capital of the Kingdom of Italy.', ['SUPPORTS']]
['The president of the United States has consistently been a male.', ['SUPPORTS']]
['Tom Hanks consistently won Academy Awards for Best Actor.', ['SUPPORTS']]
